In [1]:
! pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.3 MB/s 
     |████████████████████████████████| 453 kB 62.2 MB/s 


In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from konlpy.tag import Okt

## Task1

빈 칸을 채워주세요!

단계별 output이 github 파일에는 남아있으니 그 output과 동일한 형태인지 확인하면서 진행해주시면 됩니다~

In [21]:
#1. 생성할 문장 데이터

sentence = ("Brick walls are there for a reason and you must not think "
            "that the brick walls aren't there to keep us out, but rather "
            "in this way that the brick walls are there to show us how badly we want things.")

In [22]:
#2. 문자 집합 만들기
word_set = list(sorted(set(sentence)))

## 문제(1): 각 문자에 정수 인코딩 (공백도 하나의 원소로 포함)
vocab = {word:encode for encode, word in enumerate(word_set)}

In [23]:
print(vocab)

{' ': 0, "'": 1, ',': 2, '.': 3, 'B': 4, 'a': 5, 'b': 6, 'c': 7, 'd': 8, 'e': 9, 'f': 10, 'g': 11, 'h': 12, 'i': 13, 'k': 14, 'l': 15, 'm': 16, 'n': 17, 'o': 18, 'p': 19, 'r': 20, 's': 21, 't': 22, 'u': 23, 'w': 24, 'y': 25}


In [24]:
#3. 문자 집합 크기 확인

vocab_size = len(vocab)
print('문자 집합 크기 : {}'.format(vocab_size))

문자 집합 크기 : 26


In [25]:
#4. 하이퍼 파라미터 설정(자유롭게 수정해보세요!)
print(len(sentence))
hidden_size = vocab_size # 같아야 하는 것 확인!
sequence_length = 20  # 너무 길거나 너무 짧게 잡으면 안됩니다!
learning_rate = 0.01

198


In [26]:
#5. seqence 길이 단위 자르기

# 데이터 구성을 위한 리스트
x_data = []
y_data = []

## 문제(2): 반복문 내에서의 인덱싱을 사용하여 sequence_length 값 단위로 샘플을 잘라 데이터 만들기, y_str은 x_str은 한 칸씩 쉬프트된 sequnce

for i in range(0, len(sentence) - sequence_length):
  x_str = sentence[i:i+sequence_length]
  y_str = sentence[i+1:i+sequence_length+1]
  print(i, x_str, "->", y_str)

  # x_str과 y_str이 문자집합에 해당하는 인덱스를 각각 x_data, y_data에 append
  x_data.append([vocab[c] for c in x_str])
  y_data.append([vocab[d] for d in y_str])

0 Brick walls are ther -> rick walls are there
1 rick walls are there -> ick walls are there 
2 ick walls are there  -> ck walls are there f
3 ck walls are there f -> k walls are there fo
4 k walls are there fo ->  walls are there for
5  walls are there for -> walls are there for 
6 walls are there for  -> alls are there for a
7 alls are there for a -> lls are there for a 
8 lls are there for a  -> ls are there for a r
9 ls are there for a r -> s are there for a re
10 s are there for a re ->  are there for a rea
11  are there for a rea -> are there for a reas
12 are there for a reas -> re there for a reaso
13 re there for a reaso -> e there for a reason
14 e there for a reason ->  there for a reason 
15  there for a reason  -> there for a reason a
16 there for a reason a -> here for a reason an
17 here for a reason an -> ere for a reason and
18 ere for a reason and -> re for a reason and 
19 re for a reason and  -> e for a reason and y
20 e for a reason and y ->  for a reason and yo
21

In [27]:
# 출력해서 한 칸씩 쉬프트된 것 확인하기!

print(x_data[0])
print(y_data[0])

[4, 20, 13, 7, 14, 0, 24, 5, 15, 15, 21, 0, 5, 20, 9, 0, 22, 12, 9, 20]
[20, 13, 7, 14, 0, 24, 5, 15, 15, 21, 0, 5, 20, 9, 0, 22, 12, 9, 20, 9]


In [28]:
##6. 입력 시퀀스에 대해 원핫인코딩 수행

## 문제(4) : x_data를 원핫인코딩 > numpy의 eye를 쓸 수 있지 않을까?
x_one_hot = [np.eye(vocab_size,k=1,dtype=int)[i] for i in x_data]

##7. 입력 데이터, 레이블데이터 텐서로 변환

## 문제(5) : x_one_hot과 y_data 텐서로 변환 : 둘 다 같은 형식의 텐서로 변환하면 될까?? (FloatTensor, LongTesor 중 맞는 것은?)
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

In [29]:
##8. 크기 확인
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([178, 20, 26])
레이블의 크기 : torch.Size([178, 20])


In [30]:
##9.원핫인코딩 결과 샘플 확인하기
print(X[0])

tensor([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0.,

In [31]:
##10. 레이블 데이터 샘플 확인하기
print(Y[0])

tensor([20, 13,  7, 14,  0, 24,  5, 15, 15, 21,  0,  5, 20,  9,  0, 22, 12,  9,
        20,  9])


In [32]:
##11. RNN 모델 구현

##문제(6) : 기본 pytorch 인자 넣기 연습 + forward 채우기
### 조건 : rnn layer 2개 쌓기 + 마지막은 fc layer
### batch_first 설정 필요할까? (유튜브 강의 참고)

class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
    self.fc = torch.nn.Linear(hidden_dim, vocab_size, bias=True) # label개수 == vocab_size개수
  
  def forward(self, x):
    x,_ = self.rnn(x)
    x = self.fc(x)
    return x

In [33]:
net = Net(vocab_size, hidden_size, layers=2)

In [34]:
##12. loss function
criterion = torch.nn.CrossEntropyLoss()

##13. optimizer
optimizer = optim.Adam(net.parameters(), learning_rate)

In [35]:
##14. 출력 크기 점검
outputs = net(X)
print(outputs.shape)

torch.Size([178, 20, 26])


In [36]:
Y.view(-1).size()[0]

3560

In [39]:
##15. Training 시작

for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    ##문제(7) : outputs, Y 형태 그대로 넣으면 안되죠. view 함수를 이용해 loss값을 계산해봅시다.
    loss = criterion(outputs.view(Y.view(-1).size()[0],-1), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([word_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += word_set[result[-1]]

    print(predict_str)

          h h   h         h    h   h       h        h     h    h             h h      h     h               h          h  h  h     h    h             h h   h     h  h                       h h    h
                                                                                                                                                                                                     
                                                                                                                                                                                                     
                                                                                                                                                                                                     
                                                                                                                                                                                                     
          

In [40]:
predict_str

"rick walls are there tor a reason and you must not think that the brick walls are 't there to keep us out, but rather in this way that the brick walls are there to show us how badly we want thinksm"

In [41]:
sentence

"Brick walls are there for a reason and you must not think that the brick walls aren't there to keep us out, but rather in this way that the brick walls are there to show us how badly we want things."

In [42]:
cor = 0
for i in range(len(predict_str)):
  if list(predict_str)[i] == list(sentence)[i+1]: # sentence의 경우 B가 없으므로
    cor +=1

print(f'정확도 {cor/len(predict_str)*100:.2f}%')

정확도 97.97%


결과가 어떤가요?? 마지막 에폭의 문장이 그럴싸한가요?

## Task2

위 sentence는 제가 임의로 생성한 문장들입니다.

마음에 드시는 문구 가져오셔서 문장이 어떻게 생성되는지 확인해보세요! 

영어가 아닌 한국어로 시도해보는 것도 좋겠죠? 

수정이 많이 필요(토큰화 등) 할 수 있으나 한번 시도해보시는 것 권장드립니다 :)

위 베이스라인은 어디든 수정하셔도 좋고 조금 더 자연스러운 문장이 나올 수 있게 다양한 시도를 해보세요!

조건 : 문장 3개 이상, 연결성이 있는 문장을 " " 으로 구분하여 ( )에 넣기

In [43]:
sentence = ('젊은 피가 젊은 사랑을 후회 할 수가 있나'
            '나도 뭐가 뭔지 모르겠는데'
            '언젠가는 나도 누구의 버림을 받겠지'
            '그래도 나는 아무 상관없는 걸'
            )

In [44]:
okt = Okt()
word = okt.pos(sentence)
word

[('젊은', 'Adjective'),
 ('피', 'Noun'),
 ('가', 'Josa'),
 ('젊은', 'Adjective'),
 ('사랑', 'Noun'),
 ('을', 'Josa'),
 ('후회', 'Noun'),
 ('할', 'Verb'),
 ('수가', 'Noun'),
 ('있나나도', 'Adjective'),
 ('뭐', 'Noun'),
 ('가', 'Josa'),
 ('뭔', 'Modifier'),
 ('지', 'Modifier'),
 ('모르겠는데', 'Verb'),
 ('언젠가', 'Adverb'),
 ('는', 'Verb'),
 ('나도', 'Verb'),
 ('누구', 'Noun'),
 ('의', 'Josa'),
 ('버림을', 'Verb'),
 ('받겠지', 'Verb'),
 ('그래도', 'Adverb'),
 ('나', 'Noun'),
 ('는', 'Josa'),
 ('아무', 'Noun'),
 ('상관없는', 'Adjective'),
 ('걸', 'Noun')]

In [45]:
word = [w[0] for w in word]
vocab = {word:encode for encode, word in enumerate(sorted(set(word)))}
vocab_size = len(vocab)
print(vocab_size)
vocab # 형태소 단위이기 때문에 뛰어쓰기 포함할 필요없음

25


{'가': 0,
 '걸': 1,
 '그래도': 2,
 '나': 3,
 '나도': 4,
 '누구': 5,
 '는': 6,
 '모르겠는데': 7,
 '뭐': 8,
 '뭔': 9,
 '받겠지': 10,
 '버림을': 11,
 '사랑': 12,
 '상관없는': 13,
 '수가': 14,
 '아무': 15,
 '언젠가': 16,
 '을': 17,
 '의': 18,
 '있나나도': 19,
 '젊은': 20,
 '지': 21,
 '피': 22,
 '할': 23,
 '후회': 24}

In [76]:
hidden_size = vocab_size # 같아야 하는 것 확인!
sequence_length = 7 # 너무 길거나 너무 짧게 잡으면 안됩니다!
learning_rate = 0.01

#5. seqence 길이 단위 자르기

# 데이터 구성을 위한 리스트
x_data = []
y_data = []

## 문제(2): 반복문 내에서의 인덱싱을 사용하여 sequence_length 값 단위로 샘플을 잘라 데이터 만들기, y_str은 x_str은 한 칸씩 쉬프트된 sequnce

for i in range(0, len(word) - sequence_length):
  x_str = word[i:i+sequence_length]
  y_str = word[i+1:i+sequence_length+1]
  print(i, x_str, "->", y_str)

  # x_str과 y_str이 문자집합에 해당하는 인덱스를 각각 x_data, y_data에 append
  x_data.append([vocab[c] for c in x_str])
  y_data.append([vocab[d] for d in y_str])

0 ['젊은', '피', '가', '젊은', '사랑', '을', '후회'] -> ['피', '가', '젊은', '사랑', '을', '후회', '할']
1 ['피', '가', '젊은', '사랑', '을', '후회', '할'] -> ['가', '젊은', '사랑', '을', '후회', '할', '수가']
2 ['가', '젊은', '사랑', '을', '후회', '할', '수가'] -> ['젊은', '사랑', '을', '후회', '할', '수가', '있나나도']
3 ['젊은', '사랑', '을', '후회', '할', '수가', '있나나도'] -> ['사랑', '을', '후회', '할', '수가', '있나나도', '뭐']
4 ['사랑', '을', '후회', '할', '수가', '있나나도', '뭐'] -> ['을', '후회', '할', '수가', '있나나도', '뭐', '가']
5 ['을', '후회', '할', '수가', '있나나도', '뭐', '가'] -> ['후회', '할', '수가', '있나나도', '뭐', '가', '뭔']
6 ['후회', '할', '수가', '있나나도', '뭐', '가', '뭔'] -> ['할', '수가', '있나나도', '뭐', '가', '뭔', '지']
7 ['할', '수가', '있나나도', '뭐', '가', '뭔', '지'] -> ['수가', '있나나도', '뭐', '가', '뭔', '지', '모르겠는데']
8 ['수가', '있나나도', '뭐', '가', '뭔', '지', '모르겠는데'] -> ['있나나도', '뭐', '가', '뭔', '지', '모르겠는데', '언젠가']
9 ['있나나도', '뭐', '가', '뭔', '지', '모르겠는데', '언젠가'] -> ['뭐', '가', '뭔', '지', '모르겠는데', '언젠가', '는']
10 ['뭐', '가', '뭔', '지', '모르겠는데', '언젠가', '는'] -> ['가', '뭔', '지', '모르겠는데', '언젠가', '는', '나도']
11 ['가', '뭔', '지', '모르겠는데',

In [77]:
# 출력해서 한 칸씩 쉬프트된 것 확인하기!

print(x_data[0])
print(y_data[0])

[20, 22, 0, 20, 12, 17, 24]
[22, 0, 20, 12, 17, 24, 23]


In [78]:
##6. 입력 시퀀스에 대해 원핫인코딩 수행

## 문제(4) : x_data를 원핫인코딩 > numpy의 eye를 쓸 수 있지 않을까?
x_one_hot = [np.eye(vocab_size,k=1,dtype=int)[i] for i in x_data]

##7. 입력 데이터, 레이블데이터 텐서로 변환

## 문제(5) : x_one_hot과 y_data 텐서로 변환 : 둘 다 같은 형식의 텐서로 변환하면 될까?? (FloatTensor, LongTesor 중 맞는 것은?)
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

In [79]:
##8. 크기 확인
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([21, 7, 25])
레이블의 크기 : torch.Size([21, 7])


In [80]:
##9.원핫인코딩 결과 샘플 확인하기
print(X[0])

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 1., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.]])


In [81]:
##10. 레이블 데이터 샘플 확인하기
print(Y[0])

tensor([22,  0, 20, 12, 17, 24, 23])


In [82]:
##11. RNN 모델 구현

##문제(6) : 기본 pytorch 인자 넣기 연습 + forward 채우기
### 조건 : rnn layer 2개 쌓기 + 마지막은 fc layer
### batch_first 설정 필요할까? (유튜브 강의 참고)

class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
    self.fc = torch.nn.Linear(hidden_dim, vocab_size, bias=True) # label개수 == vocab_size개수
  
  def forward(self, x):
    x,_ = self.rnn(x)
    x = self.fc(x)
    return x

In [83]:
net = Net(vocab_size, hidden_size, layers=2)

In [84]:
##12. loss function
criterion = torch.nn.CrossEntropyLoss()

##13. optimizer
optimizer = optim.Adam(net.parameters(), learning_rate)

In [85]:
##14. 출력 크기 점검
outputs = net(X)
print(outputs.shape)

torch.Size([21, 7, 25])


In [86]:
Y.view(-1).size()[0]

147

In [87]:
##15. Training 시작

for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    ##문제(7) : outputs, Y 형태 그대로 넣으면 안되죠. view 함수를 이용해 loss값을 계산해봅시다.
    loss = criterion(outputs.view(Y.view(-1).size()[0],-1), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = "제게 "
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([word[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += ' '+word[result[-1]]

    print(predict_str)

제게 을을을을을을을 을 을 을 을 을 을 을 을 을 을 을 을 을 을 을 을 을 을 을 을
제게 을젊은젊은젊은젊은젊은젊은 을 젊은 후회 젊은 젊은 젊은 젊은 을 젊은 을 을 젊은 젊은 젊은 을 젊은 을 을 을 젊은
제게 젊은젊은젊은젊은젊은받겠지젊은 젊은 후회 후회 젊은 젊은 젊은 받겠지 젊은 젊은 후회 을 젊은 젊은 젊은 젊은 젊은 후회 젊은 젊은 후회
제게 후회젊은후회후회젊은젊은젊은 젊은 후회 후회 젊은 젊은 젊은 모르겠는데 후회 젊은 후회 을 젊은 젊은 젊은 후회 후회 후회 젊은 후회 후회
제게 후회젊은후회후회젊은후회젊은 모르겠는데 후회 후회 젊은 후회 젊은 모르겠는데 후회 후회 후회 을 후회 후회 젊은 후회 후회 후회 후회 후회 후회
제게 후회젊은후회후회후회후회후회 모르겠는데 후회 후회 젊은 후회 젊은 할 후회 후회 후회 을 후회 후회 젊은 가 후회 후회 후회 후회 후회
제게 후회젊은후회젊은후회는받겠지 모르겠는데 의 수가 젊은 후회 받겠지 할 는 후회 후회 을 후회 을 뭐 가 후회 후회 누구 모르겠는데 후회
제게 젊은젊은후회젊은는는받겠지 모르겠는데 의 수가 젊은 할 받겠지 할 는 후회 후회 을 누구 을 뭐 가 젊은 후회 누구 모르겠는데 후회
제게 젊은젊은후회젊은는는나 모르겠는데 의 수가 젊은 할 받겠지 할 는 후회 후회 을 누구 을 뭐 가 젊은 후회 누구 모르겠는데 의
제게 젊은젊은할수가는는나 모르겠는데 의 수가 젊은 있나나도 받겠지 할 는 후회 사랑 을 누구 을 뭐 가 젊은 후회 누구 모르겠는데 의
제게 젊은젊은할수가는는나 모르겠는데 의 수가 젊은 있나나도 받겠지 할 는 후회 사랑 을 누구 가 뭐 가 젊은 후회 누구 모르겠는데 의
제게 젊은젊은할받겠지는는나 모르겠는데 의 수가 젊은 있나나도 받겠지 할 는 후회 사랑 을 누구 가 뭐 가 젊은 후회 누구 모르겠는데 의
제게 젊은젊은할받겠지는는나 모르겠는데 의 수가 젊은 있나나도 받겠지 할 는 후회 사랑 을 누구 가 뭐 가 젊은 후회 사랑 모르겠는데 의
제게 젊은젊은할받겠지는는나 모르겠는데 의 수가 젊은 있나

In [88]:
predict_str

'제게 뭔젊은버림을뭔나도는나 모르겠는데 의 수가 젊은 있나나도 받겠지 할 는 후회 사랑 을 누구 가 뭐 가 젊은 후회 언젠가 지 피'

In [89]:
sentence

'젊은 피가 젊은 사랑을 후회 할 수가 있나나도 뭐가 뭔지 모르겠는데언젠가는 나도 누구의 버림을 받겠지그래도 나는 아무 상관없는 걸'

In [90]:
cor = 0
for i in range(len(sentence)):
  if list(predict_str)[i] == list(sentence)[i]: # sentence의 경우 B가 없으므로
    cor +=1

print(f'정확도 {cor/len(predict_str)*100:.2f}%')
# 같은 로직을 거쳤지만 정확도가 많이 떨어진다.

정확도 12.50%
